In [1]:
import pandas as pd
import numpy as np 
import random
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"
import ast

# Rolling Periods

In [19]:
#Finding the initial portoflio from desired investemnt universe 

def calculate_pdi(num_assets, num_crytpos, etf_tickers,crypto_tickers, weekly_returns): 
        
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/52))-1
            
            return(Result)

        pca = PCA()
        PDI_dict = {}
        samples = []
        for number in [num_assets]:
            for i in range(1,30000):
                for ii in [num_crytpos]:
                    t = random.sample(etf_tickers,number-ii)
                    crytpo = random.sample(crypto_tickers,ii)
                    t.extend(crytpo)
                    samples.append(t)
        
        seen = set()
        samples_mini = [x for x in samples if frozenset(x) not in seen and not seen.add(frozenset(x))]

        
        for i,y in tqdm(zip(samples_mini,range(1,len(samples_mini)+1))):
            #prog = int(y/len(samples_mini)*100)
            #progress_bar.progress(prog)
            #status_text.text("{}% Complete".format(prog))
            n_assets = len(i)
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            ann_ret = meanRetAn(list(port_weekly_return))
            an_cov = weekly_returns[i].cov()
            port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
            corr_matrix = np.array(weekly_returns[i].corr())
            principalComponents = pca.fit(corr_matrix)
            PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
            
            PDI_dict[y] = {}
            PDI_dict[y]["PDI_INDEX"] = PDI
            PDI_dict[y]["# of Assets"] = len(i)
            PDI_dict[y]["Assets"] = i
            PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
            PDI_dict[y]["Annual Return"] = ann_ret
            PDI_dict[y]["Annual STD"] = port_std
        

            


        PDI_DF = pd.DataFrame(PDI_dict).T
        PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
        PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
        PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
        PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
        PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
        PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)

        return PDI_DF



############################################################## Trading Strategy #################################################################################
# Trading algorithm that uses the portfolio chosen, and allocated weights accordingly
def calculate_pdi_weights( returns,return_mean_range): 

    n = len(returns.columns)
    w = [[(100/n)/100]*n]
    for i in range(1,30000):
        weights = [random.random() for _ in range(n)]
        sum_weights = sum(weights)
        weights = [1*w/sum_weights for w in weights]
        w.append(list(np.round(weights,2)))
    weights_new = []
    for i in w:
        if i not in weights_new:
            weights_new.append(i)


    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for y,num in tqdm(zip(weights_new, range(0,len(weights_new),1))):
        
        port_ret  = returns.mul(y,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = returns.cov()
        port_std = np.sqrt(np.dot(np.array(y).T, np.dot(an_cov, y)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(returns.mul(y).cov())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[num ] = {}
        PDI_dict[num ]["PDI_INDEX"] = PDI
        PDI_dict[num ]["# of Assets"] = len(y)
        PDI_dict[num ]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[num ]["Annual Return"] = ann_ret
        PDI_dict[num ]["weights"] = y
        PDI_dict[num ]["Annual STD"] = port_std

    df = pd.DataFrame(PDI_dict).T
    df["PDI_INDEX"] = df["PDI_INDEX"].astype(float)
    df["Sharpe Ratio"] = df["Sharpe Ratio"].astype(float)
    df["Annual Return"] = df["Annual Return"].astype(float)
    df["Annual STD"] = df["Annual STD"].astype(float)

    return df



############################################################## Trading Strategy #################################################################################
# Trading algorithm that finds new portfolios each quarter
def pca_per_weights_rolling(return_data, portfolio, interval, ret_range_mean, max_pdi_train, train_q4):
        data = return_data.copy() # data containing weekly returns
        tickers = list(data.columns)
        data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
        weeks_list = data.index# grabbing all index dates
        data.index = data.index.to_period(interval) # converting the index to quarterly sets
        periods = data.index.unique() # taking the unique quarters to loop

        
        

        #print(periods)
        first_period = periods[0] # the first period of the time frame
        remaining_periods = periods[1:] # the remianing periods of the time framr
        first_periods = periods[:-1] # all periods minus the last

        ########################################  Function for pdi ########## ########## ########## ########## ########### #########  
        def pdi_period(returns, period, weights):
            pca = PCA()
            corr_matrix = np.array(returns.loc[period].mul(weights).cov())
            principalComponents = pca.fit(corr_matrix)
            return 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
        ########## ########## ########## ##########  Mean Annual Return Function ########## ########## ########## ########## ########## 
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/ret_range_mean))-1
            
            return(Result)

        ########## ########## ########## ##########  Portfolio Return ########## ########## ########## ########## ########## ########## 
        def port_ret(returns, period, weights): # function for calculating returns
            portfolio_weights_ew = weights
            port_return = returns.loc[period].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            return  port_return

        pdi_performance_w = [] #saving performance of max pdi
        sharpe_performance_w= [] #saving performance of optimal portfolio of max portfolio
        equal_performance = [] #saving equal weight portflio performance 
        sharpe_2_performance_w =[]

        weights_pdi_performance_w = [] #weights for max pdi allocation over time
        weights_sharpe_performance_w= [] #weights for max sharpe ratio allocation over time
        weights_sharpe_2_performance_w= []
        weights_equal_performance = [] # weights for equal portfolio over time "same all periods"

        periods_weights = [] # saving periods for the weights allocation

        pdi_weights_pdi_performance_w = [] #pdi for max pdi allocation over time
        pdi_weights_sharpe_performance_w= [] #pdi for max sharpe ratio allocation over time
        pdi_weights_sharpe_2_performance_w= []
        pdi_weights_equal_performance = [] #pdi for equal portfolio over time "same all periods"




        
        assets = [] # store asstes for all periods
        assets.append(portfolio) # appending portfolio
        ############################################################ Calculate first period ####################################################################################
        print(first_period)
        periods_weights.append(first_period)
        first_period_df = calculate_pdi_weights( returns = train_q4[train_q4.index == "2015Q4"],return_mean_range = 12)

        ################################################## Taking the higest PDI ###########################################################################################

        id = first_period_df["PDI_INDEX"].idxmax()
        port_max_pdi_weights = first_period_df["weights"][id] # getting weights for period
        port_max_pdi_weights_pdi = first_period_df["PDI_INDEX"][id] # getting weights for period
        
        weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
        port_max_ret_period = port_ret(data, first_period,port_max_pdi_weights)  # calculating return for periods 
        pdi_performance_w.extend(port_max_ret_period) # saving return 
        pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        id_sharpe = first_period_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_weights = first_period_df["weights"][id_sharpe] # getting weights for period
        port_max_sharpe_weights_pdi = first_period_df["PDI_INDEX"][id_sharpe] # getting weights for period

        weights_sharpe_performance_w.append(port_max_sharpe_weights)   # saving weights for periods
        port_max_ret_period_sharpe = port_ret(data, first_period,port_max_sharpe_weights)  # calculating return for periods 
        sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
        pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period


        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        if len(first_period_df[first_period_df["PDI_INDEX"] > 2]) == 0:
            mini_df = first_period_df.copy()
        else:
            mini_df = first_period_df[first_period_df["PDI_INDEX"] > 2].copy()

        id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_2_weights = mini_df["weights"][id_sharpe_2] # getting weights for period
        port_max_sharpe_2_weights_pdi = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

        weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights)   # saving weights for periods
        port_max_ret_period_sharpe_2 = port_ret(data, first_period,port_max_sharpe_2_weights)  # calculating return for periods 
        sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
        pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights_pdi) # calculating pdi for first period

        ################################################################## Equal Weigths Portoflio ##########################################################################

        equal_weights = first_period_df.iloc[0]["weights"] # getting weights for period
        

        weights_equal_performance.append(equal_weights)  # saving weights for periods
        port_max_ret_period_equal = port_ret(data, first_period,equal_weights) # calculating return for periods 
        equal_performance.extend(port_max_ret_period_equal) # saving return 
        pdi_weights_equal_performance.append(max_pdi_train) # calculating pdi for first period

        ######################################################## Calculation of portfolio perfomnce #############################################################################

        for init_time, next_time in zip(first_periods, remaining_periods):
            ############ Portfolio Creation ##############################
            print("Rolling range for calculatio: {} - period of return: {}".format(init_time, next_time))
            PDI_DF = calculate_pdi_weights(returns = data.loc[init_time].dropna(axis=1), return_mean_range = ret_range_mean)
            periods_weights.append(next_time) # saving first period
            assets.append(portfolio) # appending portfolio

            ################################################## Taking the higest PDI ##########################################################################

            id = PDI_DF["PDI_INDEX"].idxmax()
            port_max_pdi_weights = PDI_DF["weights"][id] # getting weights for period
            port_max_pdi_weights_pdi = PDI_DF["PDI_INDEX"][id] # getting weights for period
            
            weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
            port_max_ret_period = port_ret(data, next_time,port_max_pdi_weights)  # calculating return for periods 
            pdi_performance_w.extend(port_max_ret_period) # saving return 
            pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            id_sharpe = PDI_DF["Sharpe Ratio"].idxmax()
            port_max_sharpe_weights = PDI_DF["weights"][id_sharpe] # getting weights for period
            port_max_sharpe_weights_pdi = PDI_DF["PDI_INDEX"][id_sharpe] # getting weights for period

            weights_sharpe_performance_w.append(port_max_sharpe_weights)   # saving weights for periods
            port_max_ret_period_sharpe = port_ret(data, next_time,port_max_sharpe_weights)  # calculating return for periods 
            sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
            pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period


            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            if len(PDI_DF[PDI_DF["PDI_INDEX"] > 2]) == 0:
                mini_df = PDI_DF.copy()
            else:
                mini_df = PDI_DF[PDI_DF["PDI_INDEX"] > 2].copy()
            id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
            port_max_sharpe_2_weights = mini_df["weights"][id_sharpe_2] # getting weights for period
            port_max_sharpe_2_weights_pdi = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

            weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights)   # saving weights for periods
            port_max_ret_period_sharpe_2 = port_ret(data, next_time,port_max_sharpe_2_weights)  # calculating return for periods 
            sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
            pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights_pdi) # calculating pdi for first period

            ################################################################## Equal Weigths Portoflio ##########################################################################

            equal_weights = PDI_DF.iloc[0]["weights"] # getting weights for period
            

            weights_equal_performance.append(equal_weights)  # saving weights for periods
            port_max_ret_period_equal = port_ret(data, next_time,equal_weights) # calculating return for periods 
            equal_performance.extend(port_max_ret_period_equal) # saving return 
            pdi_weights_equal_performance.append(pdi_period(data,next_time,equal_weights)) # calculating pdi for first period






        performance_frame = pd.DataFrame()
        performance_frame["Time"] = weeks_list
        performance_frame["Equal Weights"] = equal_performance
        performance_frame["Max PDI Weights"] = pdi_performance_w
        performance_frame["Max Sharpe Ratio Weights - PDI 2"] = sharpe_2_performance_w
        performance_frame["Max Sharpe Ratio Weights"] = sharpe_performance_w
        performance_frame["Equal Weights Cummulative"] = performance_frame["Equal Weights"].cumsum(axis=0)
        performance_frame["Max PDI Weights Cummulative"] = performance_frame["Max PDI Weights"].cumsum(axis=0) # cummulative returns max pdi
        performance_frame["Max Sharpe Ratio Weights Cummulative"] = performance_frame["Max Sharpe Ratio Weights"].cumsum(axis=0) #cummulative return sharpe ratio
        performance_frame["Max Sharpe Ratio Weights Cummulative - PDI 2"] = performance_frame["Max Sharpe Ratio Weights - PDI 2"].cumsum(axis=0) #cummulative return sharpe ratio

        weights_frame = pd.DataFrame()
        weights_frame["Period"] = periods_weights
        weights_frame["Weights Max PDI"] = weights_pdi_performance_w
        weights_frame["Weights Max Sharpe"] = weights_sharpe_performance_w
        weights_frame["Weights Max Sharpe - PDI 2"] = weights_sharpe_2_performance_w
        weights_frame["Weights Equal"] = weights_equal_performance
        weights_frame["Weights Max PDI - PDI Rolling 52"] = pdi_weights_pdi_performance_w
        weights_frame["Weights Max sharpe - PDI Rolling 52"] = pdi_weights_sharpe_performance_w
        weights_frame["Weights Max Sharpe - PDI 2 Rolling 52"] = pdi_weights_sharpe_2_performance_w
        weights_frame["Weights equal - PDI"] = pdi_weights_equal_performance
        weights_frame["Assets"] = assets







        return performance_frame, weights_frame



In [20]:
"""#Getting Data
our_uni = pd.read_csv("our_uni.csv", index_col="Ticker") # our defined universe


returns_crypto = pd.read_csv("crypto_weekly.csv" , index_col="Date")
tick_crypto = list(returns_crypto.columns)
returns_crypto.index = pd.to_datetime(returns_crypto.index)
returns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date") # loading returns dataframe
returns_weekly = returns_weekly[['IWO', 'IHI', 'VIOG', 'PSJ', 'XMHQ', 'BBP', 'ARKW', 'DWAS', 'FYC', 'FEMS', 'XTN', 'KRE', 'IYH', 'IAK', 'SIZE', 'IAT']]
etf_tick = ['IWO', 'IHI', 'VIOG', 'PSJ', 'XMHQ', 'BBP', 'ARKW', 'DWAS', 'FYC', 'FEMS', 'XTN', 'KRE', 'IYH', 'IAK', 'SIZE', 'IAT']
returns_weekly.index = pd.to_datetime(returns_weekly.index) # converting returns dataframe to datetime
returns_merge = pd.merge(returns_crypto,returns_weekly,left_index=True,right_index=True)"""

'#Getting Data\nour_uni = pd.read_csv("our_uni.csv", index_col="Ticker") # our defined universe\n\n\nreturns_crypto = pd.read_csv("crypto_weekly.csv" , index_col="Date")\ntick_crypto = list(returns_crypto.columns)\nreturns_crypto.index = pd.to_datetime(returns_crypto.index)\nreturns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date") # loading returns dataframe\nreturns_weekly = returns_weekly[[\'IWO\', \'IHI\', \'VIOG\', \'PSJ\', \'XMHQ\', \'BBP\', \'ARKW\', \'DWAS\', \'FYC\', \'FEMS\', \'XTN\', \'KRE\', \'IYH\', \'IAK\', \'SIZE\', \'IAT\']]\netf_tick = [\'IWO\', \'IHI\', \'VIOG\', \'PSJ\', \'XMHQ\', \'BBP\', \'ARKW\', \'DWAS\', \'FYC\', \'FEMS\', \'XTN\', \'KRE\', \'IYH\', \'IAK\', \'SIZE\', \'IAT\']\nreturns_weekly.index = pd.to_datetime(returns_weekly.index) # converting returns dataframe to datetime\nreturns_merge = pd.merge(returns_crypto,returns_weekly,left_index=True,right_index=True)'

In [21]:
np.random.seed(42)
random.seed(42)
#Defining training data
dicti = {"MST": ['SMH', 'SPMD', 'SLYG', 'RPV', 'XSD', 'FIW', 'XTN', 'EZM', 'EWX', 'VIOG', 'XSW', 'KIE', 'PJP', 'PRN', 'XSMO', 'KCE'], "PDI_Boot":['IWO', 'IHI', 'VIOG', 'PSJ', 'XMHQ', 'BBP', 'ARKW', 'DWAS', 'FYC', 'FEMS', 'XTN', 'KRE', 'IYH', 'IAK', 'SIZE', 'IAT']}
result = {}
for uni in list(dicti.keys()):
    result[uni] = {}
    returns_crypto = pd.read_csv("crypto_weekly.csv" , index_col="Date")
    tick_crypto = list(returns_crypto.columns)
    returns_crypto.index = pd.to_datetime(returns_crypto.index)
    returns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date") # loading returns dataframe
    returns_weekly = returns_weekly[dicti[uni]]
    etf_tick = dicti[uni]
    returns_weekly.index = pd.to_datetime(returns_weekly.index) # converting returns dataframe to datetime
    returns_merge = pd.merge(returns_crypto,returns_weekly,left_index=True,right_index=True)
    train_return = returns_merge[returns_merge.index.year <= 2015] # training on data from 2015
    train_return.index = train_return.index.to_period("Q")

    for num_c in [1,2]:
        result[uni]["num_crypto_{}".format(str(num_c))] = {}
        for dingo in ["Sharpe Ratio"]:
            result[uni]["num_crypto_{}".format(str(num_c))][dingo] = {}
            for i in [6]:
                result[uni]["num_crypto_{}".format(str(num_c))][dingo][i] = {}
            
                print("---------------------- Calculating diversification and performnce for training period -------------------------------")
                pdi_train = calculate_pdi(num_assets = i, num_crytpos=num_c, etf_tickers = etf_tick ,crypto_tickers=tick_crypto, weekly_returns = train_return) # training on data from 2015 - getting max PDI portflio

                id_index = pdi_train[dingo].idxmax() # getting index id for max pdi
                id_index_pdi = pdi_train["PDI_INDEX"][id_index]
                assets_port = ast.literal_eval(pdi_train.loc[id_index]["Assets"]) # max pdi portfolio
                print("---------------------- Selcted Portfolio -------------------------------")
                print(assets_port)
                print("------------------------------------------------------------------------")
                ini_porti = assets_port # starting portfolio 
                test_retuns = returns_merge[returns_merge.index.year >= 2016]
                
                train_return_1 = train_return
                #train_return_1.index = train_return_1.index.to_period("Q")
                train_return_1 = train_return_1[assets_port]
                test_week = test_retuns[assets_port] #returns for selected portfolio
                print("-----------------------------------  Calculating Performance of selected portfolio -----------------------------------")
                performance, weights = pca_per_weights_rolling(return_data = test_week, portfolio = ini_porti, interval = "Q", train_q4= train_return_1, ret_range_mean = 12, max_pdi_train=id_index_pdi) # running strategy for 2016 and forward
                result[uni]["num_crypto_{}".format(str(num_c))][dingo][i]["performance"] = performance
                result[uni]["num_crypto_{}".format(str(num_c))][dingo][i]["weights"] = weights
                print("----------------------------------- Done -----------------------------------")
                print("----------------------------------- Access dataframes - performance and weights  -----------------------------------")

---------------------- Calculating diversification and performnce for training period -------------------------------


20976it [01:36, 217.00it/s]


---------------------- Selcted Portfolio -------------------------------
['ARKW', 'PSJ', 'BBP', 'IAK', 'KRE', 'XVG-USD']
------------------------------------------------------------------------
-----------------------------------  Calculating Performance of selected portfolio -----------------------------------
2016Q1


29993it [01:10, 423.42it/s]


Rolling range for calculatio: 2016Q1 - period of return: 2016Q2


...

Rolling range for calculatio: 2020Q2 - period of return: 2020Q3


29987it [01:05, 454.77it/s]


Rolling range for calculatio: 2020Q3 - period of return: 2020Q4


29991it [01:05, 455.21it/s]


----------------------------------- Done -----------------------------------
----------------------------------- Access dataframes - performance and weights  -----------------------------------


In [79]:
for i in ['MST', 'PDI_Boot']:
    for ii in ["num_crypto_1","num_crypto_2"]:
        for iii in ["Sharpe Ratio"]:
            result[i][ii][iii][6]['performance'].to_csv("{}_{}_{}_performance.csv".format(i,ii,iii))
            result[i][ii][iii][6]['weights'].to_csv("{}_{}_{}_weights.csv".format(i,ii,iii))



In [22]:
px.line(result["MST"]["num_crypto_2"]["Sharpe Ratio"][6]["performance"], x="Time", y=["Equal Weights Cummulative","Max PDI Weights Cummulative","Max Sharpe Ratio Weights Cummulative","Max Sharpe Ratio Weights Cummulative - PDI 2"])


In [81]:
etf_per = pd.read_csv("MST_FIXED_weights_1_Sharpe Ratio_performance.csv")

In [82]:
eq_weights_cum = []

eq_weights_cum.append( etf_per["Time"])
eq_weights_cum.append(etf_per["Equal Weights Cummulative"])
eq_weights_cum.append(result["MST"]["num_crypto_1"]["Sharpe Ratio"][6]["performance"]["Equal Weights Cummulative"])
eq_weights_cum.append(result["MST"]["num_crypto_2"]["Sharpe Ratio"][6]["performance"]["Equal Weights Cummulative"])


In [23]:
etf_c = pd.read_csv("MST_num_crypto_1_Sharpe Ratio_performance.csv")
etf_c_2 = pd.read_csv("MST_num_crypto_2_Sharpe Ratio_performance.csv")
etf = pd.read_csv("MST_FIXED_weights_1_Sharpe Ratio_performance.csv")
etf_c = etf_c.set_index("Time").drop(columns=["Unnamed: 0"])
etf_c = etf_c.add_suffix('_1_Crypto')
etf_c_2 = etf_c_2.set_index("Time").drop(columns=["Unnamed: 0"])
etf_c_2 = etf_c_2.add_suffix('_2_Crypto')
etf = etf.set_index("Time").drop(columns=["Unnamed: 0"])
merge = pd.merge(etf_c,etf, left_index=True,right_index=True)
merge_merge = pd.merge(merge, etf_c_2,left_index=True,right_index=True)
merge_merge.rename(columns = {"Max PDI Weights Cummulative_1_Crypto":"Max PDI 1 Crypto","Max PDI Weights Cummulative_2_Crypto":"Max PDI 2 Crypto","Max PDI Weights Cummulative":"Max PDI ETF",
"Max Sharpe Ratio Weights Cummulative_1_Crypto":"Max Sharpe 1 Crypto","Max Sharpe Ratio Weights Cummulative_2_Crypto":"Max Sharpe 2 Crypto","Max Sharpe Ratio Weights Cummulative":"Max Sharpe ETF",
"Equal Weights Cummulative_1_Crypto":"1/N 1 Crypto","Equal Weights Cummulative_2_Crypto":"1/N 2 Crypto","Equal Weights Cummulative":"1/N ETF"},inplace=True)
plots = [["Max PDI 1 Crypto","Max PDI 2 Crypto","Max PDI ETF","URTH"],["Max Sharpe 1 Crypto","Max Sharpe 2 Crypto","Max Sharpe ETF","URTH"],["1/N 1 Crypto","1/N 2 Crypto","1/N ETF","URTH"]]
returns_urth = pd.read_csv("weeklyReturns.csv", index_col="Date")
returns_urth.index = pd.to_datetime(returns_urth.index)
returns_urth = returns_urth[returns_urth.index.year > 2015]
urt = pd.DataFrame(returns_urth["URTH"].cumsum(axis=0))
merge_merge["URTH"] = list(urt["URTH"])
for i in plots:
    fig = px.line(merge_merge, x=merge_merge.index , y = i)
    fig.update_layout(legend_title = "Allocation Strategy", width = 1200, height = 600)
    fig.update_yaxes(title="Cumulative Returns")

    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ))
    
    fig.update_xaxes(title="Time")
    fig.show()
    
    

In [104]:
etf_1_w

,Unnamed: 0,Period,Weights Max PDI,Weights max sharpe,Weights max sharpe -PDI >2,Weights equal,Weights max PDI - PDI,Weights max sharpe - PDI,Weights max sharpe - PDI >2 - PDI,Weights equal - PDI,Assets
0,0,2016Q1,[0.16666667 0.16666667 0.16666667 0.16666667 0...,[0.16666667 0.16666667 0.16666667 0.16666667 0...,[0.16666667 0.16666667 0.16666667 0.16666667 0...,[0.16666667 0.16666667 0.16666667 0.16666667 0...,1.251329,1.251329,1.251329,1.251329,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
1,1,2016Q2,"[0.18, 0.18, 0.13, 0.18, 0.12, 0.2]","[0.06, 0.01, 0.0, 0.09, 0.58, 0.27]","[0.06, 0.01, 0.0, 0.09, 0.58, 0.27]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.710153,1.006386,1.006386,1.826106,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
2,2,2016Q3,"[0.15, 0.17, 0.22, 0.09, 0.14, 0.23]","[0.06, 0.5, 0.41, 0.0, 0.01, 0.02]","[0.06, 0.5, 0.41, 0.0, 0.01, 0.02]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.476563,1.502588,1.502588,1.449534,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
3,3,2016Q4,"[0.33, 0.39, 0.0, 0.01, 0.2, 0.08]","[0.02, 0.0, 0.43, 0.01, 0.37, 0.17]","[0.02, 0.0, 0.43, 0.01, 0.37, 0.17]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.620874,1.023583,1.023583,1.641646,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
4,4,2017Q1,"[0.12, 0.2, 0.14, 0.08, 0.2, 0.26]","[0.18, 0.01, 0.03, 0.05, 0.04, 0.69]","[0.18, 0.01, 0.03, 0.05, 0.04, 0.69]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.299512,1.000747,1.000747,1.300860,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
5,5,2017Q2,"[0.06, 0.13, 0.22, 0.16, 0.2, 0.22]","[0.0, 0.22, 0.59, 0.02, 0.14, 0.02]","[0.0, 0.22, 0.59, 0.02, 0.14, 0.02]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.421064,1.116706,1.116706,1.970726,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
6,6,2017Q3,"[0.23, 0.2, 0.04, 0.18, 0.17, 0.19]","[0.02, 0.1, 0.43, 0.0, 0.02, 0.43]","[0.02, 0.1, 0.43, 0.0, 0.02, 0.43]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.943994,1.846470,1.846470,1.408806,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
7,7,2017Q4,"[0.31, 0.13, 0.07, 0.17, 0.12, 0.2]","[0.52, 0.01, 0.29, 0.06, 0.0, 0.12]","[0.52, 0.01, 0.29, 0.06, 0.0, 0.12]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.129647,1.115293,1.115293,1.386472,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
8,8,2018Q1,"[0.23, 0.17, 0.11, 0.2, 0.11, 0.18]","[0.44, 0.01, 0.19, 0.0, 0.01, 0.35]","[0.44, 0.01, 0.19, 0.0, 0.01, 0.35]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.468400,1.018891,1.018891,1.088628,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"
9,9,2018Q2,"[0.15, 0.18, 0.14, 0.19, 0.11, 0.22]","[0.13, 0.04, 0.64, 0.05, 0.0, 0.13]","[0.13, 0.04, 0.64, 0.05, 0.0, 0.13]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.404697,1.000151,1.000151,1.070777,"['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']"


In [24]:
etf_c_1_w = pd.read_csv("MST_num_crypto_1_Sharpe Ratio_weights.csv")
etf_c_2_w = pd.read_csv("MST_num_crypto_2_Sharpe Ratio_weights.csv")
etf_1_w = pd.read_csv("MST_FIXED_weights_1_Sharpe Ratio_weights.csv")
etf_1_w = etf_1_w.rename(columns={"Weights max PDI":"Weights Max PDI"})
#etf_c_1 = etf_c_1.set_index("Time").drop(columns=["Unnamed: 0"])
etf_c_1_w = etf_c_1_w.add_suffix('_1_Crypto')
etf_c_2_w = etf_c_2_w.add_suffix('_2_Crypto')
#etf_1 = etf_1.set_index("Time").drop(columns=["Unnamed: 0"])
g_1 = etf_c_1_w[["Period_1_Crypto","Weights Max Sharpe_1_Crypto","Assets_1_Crypto"]]
g_2 = etf_c_2_w[["Period_2_Crypto","Weights Max Sharpe_2_Crypto","Assets_2_Crypto"]]
g_0 = etf_1_w[["Period","Weights max sharpe","Assets"]]
pe_list = []
we_list = []
ass_list = []
for idx in g_1.index:
    if idx == 0:
        pe = g_1["Period_1_Crypto"][idx]
        w_new = g_1["Weights Max Sharpe_1_Crypto"][idx].replace(" ", ",")
        we = ast.literal_eval(w_new)
        ass = ast.literal_eval(g_1["Assets_1_Crypto"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
    else:
        pe = g_1["Period_1_Crypto"][idx]
        we = ast.literal_eval(g_1["Weights Max Sharpe_1_Crypto"][idx])
        ass = ast.literal_eval(g_1["Assets_1_Crypto"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
test = pd.DataFrame()
test["periods"] = pe_list
test["weights"] = we_list
test["assets"] = ass_list
test["weights"] = test["weights"]

fig = px.bar(test, x="periods", y="weights", color="assets", text="assets")
fig.update_yaxes(title = "Weighted Allocation in Portfolio")
fig.update_xaxes(title = "Period of Allocation")
fig.update_layout(showlegend = False, width = 800,height=500)
fig.show()

pe_list = []
we_list = []
ass_list = []
for idx in g_0.index:
    if idx == 0:
        pe = g_0["Period"][idx]
        w_new = g_0["Weights max sharpe"][idx].replace(" ", ",")
        we = ast.literal_eval(w_new)
        ass = ast.literal_eval(g_0["Assets"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
    else:
        pe = g_0["Period"][idx]
        we = ast.literal_eval(g_0["Weights max sharpe"][idx])
        ass = ast.literal_eval(g_0["Assets"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
    
test = pd.DataFrame()
test["periods"] = pe_list
test["weights"] = we_list
test["assets"] = ass_list
test["weights"] = test["weights"]

fig = px.bar(test, x="periods", y="weights", color="assets", text="assets")
fig.update_yaxes(title = "Weighted Allocation in Portfolio")
fig.update_xaxes(title = "Period of Allocation")
fig.update_layout(showlegend = False, width = 800,height=500)
fig.show()

pe_list = []
we_list = []
ass_list = []
for idx in g_2.index:
    if idx == 0:
        pe = g_2["Period_2_Crypto"][idx]
        w_new = g_2["Weights Max Sharpe_2_Crypto"][idx].replace(" ", ",")
        we = ast.literal_eval(w_new)
        ass = ast.literal_eval(g_2["Assets_2_Crypto"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
    else:
        pe = g_2["Period_2_Crypto"][idx]
        we = ast.literal_eval(g_2["Weights Max Sharpe_2_Crypto"][idx])
        ass = ast.literal_eval(g_2["Assets_2_Crypto"][idx])
        for i in range(0,len(we)):
            pe_list.append(pe)
        we_list.extend(we)
        ass_list.extend(ass)
test = pd.DataFrame()
test["periods"] = pe_list
test["weights"] = we_list
test["assets"] = ass_list
test["weights"] = test["weights"]

fig = px.bar(test, x="periods", y="weights", color="assets", text="assets")
fig.update_yaxes(title = "Weighted Allocation in Portfolio")
fig.update_xaxes(title = "Period of Allocation")
fig.update_layout(showlegend = False, width = 800,height=500)
fig.show()

In [40]:
for i in ['MST', 'PDI_Boot']:
    print(result[i]["num_crypto_1"]["Sharpe Ratio"][6]['weights'])

   Period                                    Weights Max PDI  \
0  2015Q1  [0.16666666666666666, 0.16666666666666666, 0.1...   
1  2015Q2                 [0.2, 0.21, 0.22, 0.15, 0.22, 0.0]   
2  2015Q3                [0.12, 0.3, 0.42, 0.09, 0.04, 0.03]   
3  2015Q4               [0.01, 0.36, 0.12, 0.26, 0.22, 0.04]   
4  2016Q1                [0.15, 0.19, 0.19, 0.18, 0.29, 0.0]   
5  2016Q2               [0.03, 0.31, 0.11, 0.01, 0.51, 0.03]   
6  2016Q3               [0.12, 0.18, 0.12, 0.26, 0.31, 0.01]   
7  2016Q4               [0.03, 0.34, 0.03, 0.49, 0.08, 0.04]   
8  2017Q1                [0.14, 0.23, 0.09, 0.25, 0.29, 0.0]   
9  2017Q2               [0.31, 0.35, 0.03, 0.24, 0.05, 0.02]   
10 2017Q3                 [0.21, 0.13, 0.22, 0.2, 0.23, 0.0]   
11 2017Q4               [0.07, 0.22, 0.17, 0.31, 0.21, 0.01]   
12 2018Q1                 [0.06, 0.09, 0.27, 0.2, 0.37, 0.0]   
13 2018Q2                [0.22, 0.01, 0.0, 0.19, 0.53, 0.05]   
14 2018Q3               [0.02, 0.42, 0.0